In [ ]:
# import required libraries
import os
from flask import Flask, request, jsonify
from openai import OpenAI
from dotenv import load_dotenv
import json
import re

In [ ]:
# Load OPEN AI Key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# Read schema prompt and few shot example files function
def load_file(path):
        with open(path, "r", encoding="utf-8") as f:
            return f.read()

In [ ]:
# Get prompt instruction with our schema prompt and few shot prompt. Also here we added instruction to generate sql
def get_prompt(user_query):
        schema   = load_file("schema_prompt_jo.txt")
        examples = load_file("few_shot_examples_jo.txt")
        prompt = f"""
        You are an AI that generates only SQL SELECT queries for Microsoft SQL Server.

        Schema:
        {schema}

        Examples:
        {examples}

        Instructions:
        - Only generate SELECT queries. Do not use INSERT, UPDATE, DELETE, DROP, TRUNCATE, or CLEAR.
        - Only respond to requests related to the database schema. If the input is unrelated, respond with:
        "This request is outside the scope of SQL SELECT generation."
        - Do not infer or apply status filters (like IsActive = 1, bCompleted = 1, bPercentage = 1, etc.) unless they are clearly mentioned in the user query.
        - If a foreign key field (e.g., lRequestID) is NULL in the main table, do not join or select related fields from the foreign table.
        - Only include related fields when the foreign key is not NULL.
        - Use JOINs or subqueries only when needed for resolving foreign keys or applying filters.
        - If a column name contains or ends with "ID", use a JOIN to return the corresponding human-readable value from the related table.
        - Always include foreign key ID columns from joined tables in the SELECT statement.
        - Also include the associated human-readable fields (e.g., names or descriptions) for each ID column and it represent description column Ex: StatueId represent status , using appropriate JOINs based on the schema.
        - Do not omit foreign key ID columns from joined tables. Always include both the ID and its readable counterpart.

        - Always format the column names in proper English using aliases (use `AS`) in the result. For example:
        - `FirstName` → `First Name`
        - `lSalesPersonID` → `Sales Person Name` (via join)
        - Use only column names from the schema. Do not guess or invent columns.
        - “JO” or “contracts” refers to JO Contracts. If your ask regarding status you should should return staus also.
        - Customer name: concat company_name, firstname, lastname, city with ' - ' separator. Skip separator for null fields.

        
        Goal:
        Return clean, user-friendly results suitable for reports or dashboards.
        Do not include any comments in the SQL output. This includes inline comments (--) and block comments (/* */). The SQL must be completely free of any comments.

        User: {user_query}
        SQL:
        """
        return prompt

In [ ]:
# Generate_NL_TO_SQL_QUERY
# Here we call get_prompt function and call OPEN AI API key using 'gpt-4o' model to generate natural language to sql query
# We use Microsoft sql server database 

def Generate_NL_TO_SQL_QUERY(user_query):
    prompt = get_prompt(user_query)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=500
    )
    
    token_usage = {
        'prompt_tokens': response.usage.prompt_tokens,
        'completion_tokens': response.usage.completion_tokens,
        'total_tokens': response.usage.total_tokens
    }
    
    print(token_usage)
    ai_res = response.choices[0].message.content.strip()

    print("gpt_res", ai_res )
    if  "outside the scope"  in ai_res:
        data = {"msg": ai_res, "status": 2, "token_usage":token_usage,"query": ""}
    else:
        ai_res = re.sub(r"^```sql\s*|```$", "", ai_res.strip(), flags=re.IGNORECASE | re.MULTILINE)
        
        data = {"msg": "success", "status": 1, "token_usage":token_usage,"query": ai_res.strip()}
       
    return json.dumps(data, default=str)

In [6]:
user_query = input("Enter your query?")
user_query

'list of jo contracts'

In [ ]:
generated_query = Generate_NL_TO_SQL_QUERY(user_query)

{'prompt_tokens': 3621, 'completion_tokens': 376, 'total_tokens': 3997}
gpt_res SELECT
  jc.lJoContractID AS [Contract ID],
  jc.lCustomerID AS [Customer ID],
  CASE 
    WHEN jc.lRequestID IS NOT NULL THEN dbo.GetLead_FullName(jc.lRequestID) 
    ELSE COALESCE(c.sCompany + ' - ', '') + COALESCE(c.FirstName + ' ', '') + COALESCE(c.LastName, '') + ' ' + COALESCE(c.sCity, '') 
  END AS [Customer Name],
  jc.JobName AS [Job Name],
  jc.lSalesPersonID AS [Sales Person ID],
  CONCAT(sp.sFirstName, ' ', sp.sLastName) AS [Sales Person Name],
  jc.lProjectManagerID AS [Project Manager ID],
  CONCAT(pm.sFirstName, ' ', pm.sLastName) AS [Project Manager Name],
  jc.dStart AS [Start Date],
  jc.dEnd AS [End Date],
  jc.IsActive AS [Is Active],
  jc.lRequestID AS [Request ID],
  jc.lScheduleStatusTypeID AS [Schedule Status Type ID],
  sst.sScheduleStatusType AS [Schedule Status Type],
  CASE 
    WHEN jc.lJoStatus IN (0, 1) THEN 'Quoted' 
    WHEN jc.lJoStatus = 2 THEN 'Sold' 
    ELSE 'Completed'

In [ ]:
# Example questions
# List of active contracts
# Fint out total payment received for last year only
# How many contracts generated by leads?
# How many contracts generated by customers?
# how many JoContractHaulingProduct that are in Sold?
# how many Customer added in Multiple  Routes Assigned  from JoWorksheetCustomers